In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
def extract_x(file):
    
    df = pd.read_csv(file)
    
    if file == 'epl.csv':
        x = df.drop(['fixture_date', 'fixture_id', 'Home Team', 'Away Team', 'Full Time Result',
                    'Result No Draw', 'A Win', 'B Win'], axis=1)
        x = pd.get_dummies(x, columns=["Promoted"], prefix="Promoted")

    elif file == 'nba.csv':
        x = df.drop([ 'Home Win', 'MATCH_DATE', 'HOME_TEAM', 'AWAY_TEAM', 
                                        'A Win', 'B Win'], axis=1)
        
    else:
        x = df.drop(['FIGHTER_A', 'FIGHTER_B', 'Result', 'A Win', 'B Win', 'A_CUMULATIVE_RDS', 'B_CUMULATIVE_RDS'], axis=1)
    
    return x

In [5]:
def extract_y(file):
    
    df = pd.read_csv(file)
    
    if file == 'epl.csv':
        y = df['Result No Draw']
    elif file == 'nba.csv':
        y = df['Home Win']
    else:
        y = df['Result']
    
    return y

In [6]:
def extract_odds(file):
    
    df = pd.read_csv(file)
    odds = df[['A Win', 'B Win']]
    
    return odds

In [7]:
def preprocess_data(data):
    
    min_max_scaler = preprocessing.MinMaxScaler()
    
    data = min_max_scaler.fit_transform(data)
    
    return data

In [8]:
def generate_model(sport):
    
    if sport == 'epl':
        x_train, x_test, y_train, y_test = train_test_split(epl_x, epl_y, test_size = 0.2, shuffle=False)
    elif sport == 'nba':
        x_train, x_test, y_train, y_test = train_test_split(nba_x, nba_y, test_size = 0.2, shuffle=False)
    elif sport == 'ufc':
        x_train, x_test, y_train, y_test = train_test_split(ufc_x, ufc_y, test_size = 0.2, shuffle=False)
    else:
        pass
    
    model = Sequential()
    if sport == 'epl':
        model.add(Dense(17, input_shape=(17,), activation='relu'))
        model.add(Dense(4, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        
    elif sport == 'nba':
        model.add(Dense(24, input_shape=(24,), activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
    
    elif sport == 'ufc':
        model.add(Dense(30, input_shape=(30,), activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
    
    else:
        pass

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [9]:
def testing_split(sport):
    
    if sport == 'epl':
        x_train, x_test, y_train, y_test = train_test_split(epl_x, epl_y, test_size = 0.2, shuffle=False)
    elif sport == 'nba':
        x_train, x_test, y_train, y_test = train_test_split(nba_x, nba_y, test_size = 0.2, shuffle=False)
    elif sport == 'ufc':
        x_train, x_test, y_train, y_test = train_test_split(ufc_x, ufc_y, test_size = 0.2, shuffle=False)
    else:
        pass
    
    return x_train, x_test, y_train, y_test

In [10]:
def calc_basic_returns(row):
    
    if row['predicted'] == row['actual']:
        if row['predicted'] == 1:
            return (100 * row['A Win'])
        else:
            return (100 * row['B Win'])
    else:
        return 0

In [11]:
def calc_returns_70(row):
    
    if row['A Win Prob'] > 0.70:
        if row['predicted'] == row['actual']:
            return (100 * row['A Win'])
        else:
            return 0
    elif row['B Win Prob'] > 0.70:
        if row['predicted'] == row['actual']:
            return (100 * row['B Win'])
        else:
            return 0
    else:
        return np.nan

In [12]:
def underdog_only(row):
    
    if row['predicted'] == 1:
        if row['A Win'] > row['B Win']:
            if row['predicted'] == row['actual']:
                return (100 * row['A Win'])
            else:
                return 0
        else:
            return np.nan
    else:
        if row['B Win'] > row['A Win']:
            if row['predicted'] == row['actual']:
                return (100 * row['B Win'])
            else:
                return 0
        else:
            return np.nan

In [13]:
def unit_strategy(row):
    
    if row['predicted'] == row['actual']:
        if row['predicted'] == 1:
            if row['A Win Prob'] > 0.7:
                return 200 * row['A Win']
            elif row['A Win Prob'] > 0.6:
                return 100 * row['A Win']
            else:
                return 50 * row['A Win']
        else:
            if row['B Win Prob'] > 0.7:
                return 200 * row['B Win']
            elif row['B Win Prob'] > 0.6:
                return 100 * row['B Win']
            else:
                return 50 * row['B Win']
    else:
        return 0

In [14]:
def unit_bet_calc(row):
    
    if row['A Win Prob'] > 0.7 or row['B Win Prob'] > 0.7:
        return 200
    elif row['A Win Prob'] > 0.6 or row['B Win Prob'] > 0.6:
        return 100
    else:
        return 50

In [15]:
def df_pred_processing(row):
    
    if row['A Win Prob'] >= 0.5:
        return 1
    else:
        return 0

In [16]:
def b_win_prob(row):
    
    return 1 - row['A Win Prob']

In [17]:
def predictions_dataframe(results, predicted_values, odds):
    
    df = pd.DataFrame(data={'actual': results[3], 'A Win Prob': predicted_values})
    
    df = df.join(odds)
    
    df['predicted'] = df.apply (lambda row: df_pred_processing(row), axis=1)
    df['B Win Prob'] = df.apply( lambda row: b_win_prob(row), axis=1)
    df['returns'] = df.apply (lambda row: calc_basic_returns(row), axis=1)
    df['returns_70pct'] = df.apply (lambda row: calc_returns_70(row), axis=1)
    df['returns_dogs'] = df.apply (lambda row: underdog_only(row), axis=1)
    df['returns_units'] = df.apply (lambda row: unit_strategy(row), axis=1)
    df['bets_units'] = df.apply (lambda row: unit_bet_calc(row), axis=1)
    
    return df

In [18]:
def betting_summary(df):
    
    base_returns = df['returns'].sum()
    total_bet_base = len(df.index) * 100
    profit_base = base_returns - total_bet_base
    
    returns_70 = df['returns_70pct'].sum()
    total_bets_70 = (len(df.index) - df['returns_70pct'].isnull().sum()) * 100
    profit_70 = returns_70 - total_bets_70
    
    
    returns_dogs = df['returns_dogs'].sum()
    total_bets_dogs = (len(df.index) - df['returns_dogs'].isnull().sum()) * 100
    profit_dogs = returns_dogs - total_bets_dogs
    
    returns_units = df['returns_units'].sum()
    total_bets_units = df['bets_units'].sum()
    profit_units = returns_units - total_bets_units
    
    print('Basic Strategy - Returns:', base_returns, 'Total Bet:', total_bet_base, 'ROI%:', (profit_base / total_bet_base) * 100) 
    print('70pct Strategy - Returns:', returns_70, 'Total Bet:', total_bets_70, 'ROI%:', (profit_70 / total_bets_70) * 100)
    print('Dogs Strategy - Returns:', returns_dogs, 'Total Bet:', total_bets_dogs, 'ROI%', (profit_dogs / total_bets_dogs) * 100)
    print('Units Stategy - Returns:', returns_units, 'Total Bet:', total_bets_units, 'ROI%', (profit_units / total_bets_units) * 100)

In [19]:
epl_x = extract_x('epl.csv')
epl_y = extract_y('epl.csv')
epl_odds = extract_odds('epl.csv')

nba_x = extract_x('nba.csv')
nba_y = extract_y('nba.csv')
nba_odds = extract_odds('nba.csv')

ufc_x = extract_x('ufc.csv')
ufc_y = extract_y('ufc.csv')
ufc_odds = extract_odds('ufc.csv')

In [20]:
epl_x = preprocess_data(epl_x)
nba_x = preprocess_data(nba_x)
ufc_x = preprocess_data(ufc_x)

In [21]:
epl_model = generate_model('epl')
nba_model = generate_model('nba')
ufc_model = generate_model('ufc')

2022-08-09 03:03:16.945183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
epl_split = testing_split('epl')
nba_split = testing_split('nba')
ufc_split = testing_split('ufc')

In [23]:
epl_model.fit(x= epl_split[0],
              y = epl_split[2],
              epochs=10,
              initial_epoch=0)

Epoch 1/10
57/57 [==============================] - 1s 1ms/step - loss: 0.7057 - accuracy: 0.5376
Epoch 2/10
57/57 [==============================] - 0s 990us/step - loss: 0.6796 - accuracy: 0.5631
Epoch 3/10
57/57 [==============================] - 0s 963us/step - loss: 0.6652 - accuracy: 0.6206
Epoch 4/10
57/57 [==============================] - 0s 989us/step - loss: 0.6533 - accuracy: 0.6388
Epoch 5/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6451 - accuracy: 0.6338
Epoch 6/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6397 - accuracy: 0.6405
Epoch 7/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6364 - accuracy: 0.6449
Epoch 8/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6333 - accuracy: 0.6433
Epoch 9/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6319 - accuracy: 0.6471
Epoch 10/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6296 - accuracy: 0.6449


In [24]:
nba_model.fit(x= nba_split[0],
              y = nba_split[2],
              epochs=10,
              initial_epoch=0)

Epoch 1/10
190/190 [==============================] - 1s 1ms/step - loss: 0.6752 - accuracy: 0.5762
Epoch 2/10
190/190 [==============================] - 0s 987us/step - loss: 0.6634 - accuracy: 0.6033
Epoch 3/10
190/190 [==============================] - 0s 918us/step - loss: 0.6563 - accuracy: 0.6170
Epoch 4/10
190/190 [==============================] - 0s 968us/step - loss: 0.6510 - accuracy: 0.6199
Epoch 5/10
190/190 [==============================] - 0s 920us/step - loss: 0.6478 - accuracy: 0.6222
Epoch 6/10
190/190 [==============================] - 0s 877us/step - loss: 0.6444 - accuracy: 0.6276
Epoch 7/10
190/190 [==============================] - 0s 882us/step - loss: 0.6419 - accuracy: 0.6329
Epoch 8/10
190/190 [==============================] - 0s 966us/step - loss: 0.6427 - accuracy: 0.6354
Epoch 9/10
190/190 [==============================] - 0s 914us/step - loss: 0.6408 - accuracy: 0.6286
Epoch 10/10
190/190 [==============================] - 0s 876us/step - loss: 0.6396 

In [25]:
ufc_model.fit(x= ufc_split[0],
              y = ufc_split[2],
              epochs=10,
              initial_epoch=0)

Epoch 1/10
109/109 [==============================] - 0s 958us/step - loss: 0.6989 - accuracy: 0.5127
Epoch 2/10
109/109 [==============================] - 0s 913us/step - loss: 0.6881 - accuracy: 0.5512
Epoch 3/10
109/109 [==============================] - 0s 886us/step - loss: 0.6842 - accuracy: 0.5630
Epoch 4/10
109/109 [==============================] - 0s 896us/step - loss: 0.6817 - accuracy: 0.5616
Epoch 5/10
109/109 [==============================] - 0s 916us/step - loss: 0.6777 - accuracy: 0.5713
Epoch 6/10
109/109 [==============================] - 0s 986us/step - loss: 0.6758 - accuracy: 0.5794
Epoch 7/10
109/109 [==============================] - 0s 979us/step - loss: 0.6714 - accuracy: 0.5834
Epoch 8/10
109/109 [==============================] - 0s 873us/step - loss: 0.6687 - accuracy: 0.5860
Epoch 9/10
109/109 [==============================] - 0s 912us/step - loss: 0.6655 - accuracy: 0.5866
Epoch 10/10
109/109 [==============================] - 0s 896us/step - loss: 0.664

In [26]:
epl_results = epl_model.evaluate(x=epl_split[1], y=epl_split[3])
nba_results = nba_model.evaluate(x=nba_split[1], y=nba_split[3])
ufc_results = ufc_model.evaluate(x=ufc_split[1], y=ufc_split[3])

28/28 [==============================] - 0s 804us/step - loss: 0.6714 - accuracy: 0.5742


In [27]:
epl_pred = epl_model.predict(epl_split[1])
nba_pred = nba_model.predict(nba_split[1])
ufc_pred = ufc_model.predict(ufc_split[1])

28/28 [==============================] - 0s 697us/step


In [28]:
predicted_values_epl = []
predicted_values_nba = []
predicted_values_ufc = []

for i in epl_pred:
    predicted_values_epl.append(i[0])
for i in nba_pred:
    predicted_values_nba.append(i[0])
for i in ufc_pred:
    predicted_values_ufc.append(i[0])

In [29]:
epl_predictions = predictions_dataframe(epl_split, predicted_values_epl, epl_odds)
nba_predictions = predictions_dataframe(nba_split, predicted_values_nba, nba_odds)
ufc_predictions = predictions_dataframe(ufc_split, predicted_values_ufc, ufc_odds)

In [30]:
epl_predictions

,actual,A Win Prob,A Win,B Win,predicted,B Win Prob,returns,returns_70pct,returns_dogs,returns_units,bets_units
1808,1,0.642550,1.58,2.43,1,0.357450,158.0,NaN,NaN,158.0,100
1809,1,0.381204,3.02,1.40,0,0.618796,0.0,NaN,NaN,0.0,100
1810,1,0.393550,4.14,1.25,0,0.606450,0.0,NaN,NaN,0.0,100
1811,0,0.387272,2.64,1.50,0,0.612728,150.0,NaN,NaN,150.0,100
1812,0,0.298491,4.79,1.20,0,0.701509,120.0,120.0,NaN,240.0,200
...,...,...,...,...,...,...,...,...,...,...,...
2255,1,0.723949,1.16,5.61,1,0.276051,116.0,116.0,NaN,232.0,200
2256,1,0.726811,1.44,2.87,1,0.273189,144.0,144.0,NaN,288.0,200
2257,1,0.428749,2.84,1.45,0,0.571251,0.0,NaN,NaN,0.0,50
2258,1,0.370230,1.72,2.17,0,0.629770,0.0,NaN,0.0,0.0,100


In [31]:
nba_predictions

,actual,A Win Prob,A Win,B Win,predicted,B Win Prob,returns,returns_70pct,returns_dogs,returns_units,bets_units
6053,1,0.730362,1.13,6.25,1,0.269638,113.0,113.0,NaN,226.0,200
6054,0,0.320997,7.25,1.11,0,0.679003,111.0,NaN,NaN,111.0,100
6055,0,0.244528,7.00,1.11,0,0.755472,111.0,111.0,NaN,222.0,200
6056,1,0.551230,1.57,2.55,1,0.448770,157.0,NaN,NaN,78.5,50
6057,0,0.402550,2.40,1.63,0,0.597450,163.0,NaN,NaN,81.5,50
...,...,...,...,...,...,...,...,...,...,...,...
7562,1,0.499637,1.80,2.05,0,0.500363,0.0,NaN,0.0,0.0,50
7563,0,0.594564,2.45,1.61,1,0.405436,0.0,NaN,0.0,0.0,50
7564,1,0.780305,1.36,3.40,1,0.219695,136.0,136.0,NaN,272.0,200
7565,0,0.665631,1.25,4.30,1,0.334369,0.0,NaN,NaN,0.0,100


In [32]:
ufc_predictions

,actual,A Win Prob,A Win,B Win,predicted,B Win Prob,returns,returns_70pct,returns_dogs,returns_units,bets_units
3476,1,0.505491,1.34,3.45,1,0.494509,134.0,NaN,NaN,67.0,50
3477,1,0.588823,1.59,2.50,1,0.411177,159.0,NaN,NaN,79.5,50
3478,0,0.393491,1.42,3.00,0,0.606509,300.0,NaN,300.0,300.0,100
3479,1,0.410775,3.50,1.33,0,0.589225,0.0,NaN,NaN,0.0,50
3480,1,0.434285,1.43,2.90,0,0.565715,0.0,NaN,0.0,0.0,50
...,...,...,...,...,...,...,...,...,...,...,...
4340,0,0.432776,1.87,1.95,0,0.567224,195.0,NaN,195.0,97.5,50
4341,0,0.423409,3.43,1.35,0,0.576591,135.0,NaN,NaN,67.5,50
4342,1,0.358375,1.53,2.65,0,0.641625,0.0,NaN,0.0,0.0,100
4343,0,0.440417,1.61,2.45,0,0.559583,245.0,NaN,245.0,122.5,50


In [33]:
betting_summary(epl_predictions)

Basic Strategy - Returns: 42762.0 Total Bet: 45200 ROI%: -5.393805309734513
70pct Strategy - Returns: 11140.0 Total Bet: 11500 ROI%: -3.130434782608696
Dogs Strategy - Returns: 8744.0 Total Bet: 10600 ROI% -17.50943396226415
Units Stategy - Returns: 45902.5 Total Bet: 48000 ROI% -4.369791666666667


In [34]:
betting_summary(nba_predictions)

Basic Strategy - Returns: 145045.0 Total Bet: 151400 ROI%: -4.197490092470278
70pct Strategy - Returns: 22800.0 Total Bet: 24000 ROI%: -5.0
Dogs Strategy - Returns: 31560.0 Total Bet: 35400 ROI% -10.847457627118644
Units Stategy - Returns: 134292.0 Total Bet: 139700 ROI% -3.8711524695776665


In [35]:
betting_summary(ufc_predictions)

Basic Strategy - Returns: 89939.0 Total Bet: 86900 ROI%: 3.4971231300345225
70pct Strategy - Returns: 8526.0 Total Bet: 5700 ROI%: 49.57894736842105
Dogs Strategy - Returns: 35987.0 Total Bet: 33700 ROI% 6.786350148367952
Units Stategy - Returns: 73079.0 Total Bet: 66800 ROI% 9.399700598802395
